In [ ]:
%%capture
# To run this notebook on your account, please upload the exact .zip file
# provided in the assignment sheet to your google drive storage

# Mount drive and extract the provided .zip file
from google.colab import drive
drive.mount("/content/drive")
!unzip /content/drive/MyDrive/assignment_data_bdd.zip -d /content/data

### First, let's restructure our dataset directory. This is done to avoid long path names, as well as to adjust for the lack of labels for the provided set of test images

In [ ]:
# Create a folder called images in /content/data and move train and val to that
# directory. Since test has no labels and is not used in the evaluation stage,
# it is best to delete the folder entirely
!mkdir /content/data/images
!mv /content/data/bdd100k_images_100k/bdd100k/images/100k/train \
/content/data/images
!mv /content/data/bdd100k_images_100k/bdd100k/images/100k/val \
/content/data/images
!rm -rf /content/data/bdd100k_images_100k

# Move labels folder and remove parent directories
# !mkdir /content/data/labels
!mv /content/data/bdd100k_labels_release/bdd100k/labels /content/data
!mv /content/data/labels /content/data/labels_json
!rm -rf /content/data/bdd100k_labels_release

# Rename images/val to images/test and corresponding label file
!mv /content/data/images/val /content/data/images/test
!mv /content/data/labels_json/bdd100k_labels_images_val.json \
/content/data/labels_json/bdd100k_labels_images_test.json


In [ ]:
# Just to confirm that the train dataset has 70k jpg images
!find /content/data/images/train -type f | sed -n 's/.*\.\([^.]*\)$/\1/p' | sort | uniq -c
# Also, let's install ijson to lazily load the huge json files
!pip install ijson

  70000 jpg
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.3/148.3 kB 5.9 MB/s eta 0:00:00


### Now let's directly generate our yolo-ready labels and split the given train dataset into train and val. Let's start easy with the test set. When splitting the train dataset, let's send every 10th image to val to ensure that 10 percent of the training data is set aside as a validation set.

In [ ]:
# Since we already know the classes from analysis time, let's just
# hardcode a dict to avoid counting everything again
category_indices = {
    "traffic sign": 0,
    "traffic light": 1,
    "car": 2,
    "rider": 3,
    "motor": 4, # Motorcycle, haha
    "person": 5,
    "bus": 6,
    "truck": 7,
    "bike": 8,
    "train": 9
}
# Also, the image size is fixed for the whole dataset
IMAGE_WIDTH = 1280
IMAGE_HEIGHT = 720

In [ ]:
# Generate YOLO style .txt labels for the test set
!mkdir /content/data/labels && mkdir /content/data/labels/test
import ijson
from pathlib import Path

test_labels_json_path = Path("/content/data/labels_json/bdd100k_labels_images_test.json")
test_labels_txt_root = Path("/content/data/labels/test")
with test_labels_json_path.open("r", encoding="utf-8") as test_labels_json_f:
    for obj in ijson.items(test_labels_json_f, "item"):
        for label in obj["labels"]:
            category = label["category"]
            if category in category_indices.keys():
                x1 = label["box2d"]["x1"]
                y1 = label["box2d"]["y1"]
                x2 = label["box2d"]["x2"]
                y2 = label["box2d"]["y2"]
                x_c = (x1 + x2) / (2 * IMAGE_WIDTH)
                y_c = (y1 + y2) / (2 * IMAGE_HEIGHT)
                width = (x2 - x1) / IMAGE_WIDTH
                height = (y2 - y1) / IMAGE_HEIGHT
                label_path = test_labels_txt_root / (obj["name"][:-4] + ".txt")
                with label_path.open("a", encoding="utf-8") as label_f:
                    label_f.write(f"{category_indices[category]} {x_c:.6f} {y_c:.6f} {width:.6f} {height:.6f}\n")

In [ ]:
# Generate YOLO style .txt labels for the train and val set
# Please note: At this point, there is only a "train" split, and I am splitting
# it into "train" and "val" by sending every 10th item to val
!mkdir /content/data/labels/train && mkdir /content/data/labels/val && mkdir /content/data/images/val
import ijson
from pathlib import Path
import shutil
import os

train_labels_json_path = Path("/content/data/labels_json/bdd100k_labels_images_train.json")
train_labels_txt_root = Path("/content/data/labels/train")
val_labels_txt_root = Path("/content/data/labels/val")


index = 1 # To send the 10th image and .txt label to val
with train_labels_json_path.open("r", encoding="utf-8") as train_labels_json_f:
    for obj in ijson.items(train_labels_json_f, "item"):
        image_path = Path("/content/data/images/train") / obj["name"]
        valid_labels = 0
        for label in obj["labels"]:
            category = label["category"]
            if category in category_indices.keys():
                valid_labels += 1
                x1 = label["box2d"]["x1"]
                y1 = label["box2d"]["y1"]
                x2 = label["box2d"]["x2"]
                y2 = label["box2d"]["y2"]
                x_c = (x1 + x2) / (2 * IMAGE_WIDTH)
                y_c = (y1 + y2) / (2 * IMAGE_HEIGHT)
                width = (x2 - x1) / IMAGE_WIDTH
                height = (y2 - y1) / IMAGE_HEIGHT
                # Every 10th item goes to val
                if index % 10 == 0:
                    label_path = val_labels_txt_root / (obj["name"][:-4] + ".txt")
                else:
                    label_path = train_labels_txt_root / (obj["name"][:-4] + ".txt")
                with label_path.open("a", encoding="utf-8") as label_f:
                    label_f.write(f"{category_indices[category]} {x_c:.6f} {y_c:.6f} {width:.6f} {height:.6f}\n")
        if valid_labels == 0:
            # Delete the image in train and continue
            if os.path.exists("/content/data/images/train/" + obj["name"]):
                os.remove("/content/data/images/train/" + obj["name"])
            continue
        if index % 10 == 0:
            try:
                shutil.move(image_path, Path("/content/data/images/val"))
            except Exception as e:
                # WARNING! The below print statement could crash your system!
                print(f"Encountered exception for file {obj['name']} at index: {index}. Details: {e}")
        index += 1

In [ ]:
!echo "Number of files in images/train:" $(ls -1 /content/data/images/train | wc -l)
!echo "Number of files in images/test:" $(ls -1 /content/data/images/test | wc -l)
!echo "Number of files in images/val:" $(ls -1 /content/data/images/val | wc -l)
!echo "Number of files in labels/train:" $(ls -1 /content/data/labels/train | wc -l)
!echo "Number of files in labels/test:" $(ls -1 /content/data/labels/test | wc -l)
!echo "Number of files in labels/val:" $(ls -1 /content/data/labels/val | wc -l)

Number of files in images/train: 63014
Number of files in images/test: 10000
Number of files in images/val: 6986
Number of files in labels/train: 62877
Number of files in labels/test: 10000
Number of files in labels/val: 6986


In [ ]:
# Generate YAML file
!touch /content/data/assignment.yaml
import yaml

yaml_dict = {
    "path": "/content/data",
    "train": "images/train",
    "val": "images/val",
    "test": "images/test",
    "names": {
        value: key for key, value in category_indices.items()
    }
}

with open("/content/data/assignment.yaml", "w") as yaml_f:
    yaml.dump(yaml_dict, yaml_f)


In [ ]:
# Now install ultralytics and wandb
!pip install wandb
!pip install ultralytics

In [ ]:
WANDB_KEY="b03186a24df5a7d70eee8c36ba92b3aa43d537f7"

In [ ]:
from ultralytics import YOLO
import wandb

wandb.login(key=WANDB_KEY)

model = YOLO("yolo11n.pt")
results = model.train(
    data="/content/data/assignment.yaml",
    epochs=5,
    imgsz=640,
    project="bosch-assignment",
    name="Run1-22.09",
    exist_ok=True
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/assignment.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=Run1-22.09, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, po